In [10]:
"""
Clean and simple Keras implementation of network architectures described in:
    - (ResNet-50) [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf).
    - (ResNeXt-50 32x4d) [Aggregated Residual Transformations for Deep Neural Networks](https://arxiv.org/pdf/1611.05431.pdf).
    
Python 3.
"""

from keras import layers
from keras import models
from keras.models import Sequential, Model



#
# network params
#

cardinality = 1
from keras import backend as K

def add_common_layers(y):
    y = layers.BatchNormalization()(y)
    y = layers.LeakyReLU()(y)

    return y

def grouped_convolution(y, nb_channels, _strides):
    # when `cardinality` == 1 this is just a standard convolution
    if cardinality == 1:
        return layers.Conv3D(nb_channels, kernel_size=(3, 3, 3), strides=_strides, padding='same')(y)

    assert not nb_channels % cardinality
    _d = nb_channels // cardinality

    # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
    # and convolutions are separately performed within each group
    groups = []
    for j in range(cardinality):
        group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
        groups.append(layers.Conv3D(_d, kernel_size=(3, 3, 3), strides=_strides, padding='same')(group))

    # the grouped convolutional layer concatenates them as the outputs of the layer
    y = layers.concatenate(groups)

    return y

def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1, 1), _project_shortcut=False):
    """
    Our network consists of a stack of residual blocks. These blocks have the same topology,
    and are subject to two simple rules:
    - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
    - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
    """
    shortcut = y
    
    # we modify the residual building block as a bottleneck design to make the network more economical
    y = layers.Conv3D(nb_channels_in, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same')(y)
    y = add_common_layers(y)

    # ResNeXt (identical to ResNet when `cardinality` == 1)
    y = grouped_convolution(y, nb_channels_in, _strides=_strides)
    y = add_common_layers(y)

    y = layers.Conv3D(nb_channels_out, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same')(y)
    # batch normalization is employed after aggregating the transformations and before adding to the shortcut
    y = layers.BatchNormalization()(y)

    # identity shortcuts used directly when the input and output are of the same dimensions
    if _project_shortcut or _strides != (1, 1, 1):
        # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
        # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
        shortcut = layers.Conv3D(nb_channels_out, kernel_size=(1, 1, 1), strides=_strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    y = layers.add([shortcut, y])

    # relu is performed right after each batch normalization,
    # expect for the output of the block where relu is performed after the adding to the shortcut
    y = layers.LeakyReLU()(y)

    return y
    
def residual_encoder(x):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    # pad to 64,64,64
    x=layers.ZeroPadding3D(padding=((7,8),(3,3),(8,9)))(x)
    
    # conv1
    x = layers.Conv3D(32, kernel_size=(7, 7, 7), strides=(2, 2, 2), padding='same')(x)
    x = add_common_layers(x)

    # conv2
#     x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)
    for i in range(3):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)

    # conv3
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32,_project_shortcut=project_shortcut)

    # conv4
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)
    for i in range(6):
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)

    # conv5
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)
    for i in range(3):
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)

    #x = layers.GlobalAveragePooling3D()(x)
    #x = layers.Dense(100)(x)

    return x

def residual_decoder(x):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    # conv5
    for i in range(3):
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)
    x = layers.UpSampling3D((2, 2, 2))(x)


    # conv4
    for i in range(6):
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)
    x = layers.UpSampling3D((2, 2, 2))(x)

    # conv3
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        #strides = (2, 2, 2) if i == 0 else (1, 1, 1)
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32,_project_shortcut=project_shortcut)
    x = layers.UpSampling3D((2, 2, 2))(x)        
    
    # conv2
    for i in range(3):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 32, 32, _project_shortcut=project_shortcut)
#     x = layers.UpSampling3D((2, 2, 2))(x)

    # conv1    
    x = layers.Conv3D(1, kernel_size=(7, 7, 7), padding='same')(x)
    x = add_common_layers(x)
    x = layers.UpSampling3D((2, 2, 2))(x)
    
    x=layers.Cropping3D(cropping=((7,8),(3,3),(8,9)))(x)
    return x

#
# image dimensions
#

img_x = 49
img_y = 58
img_z = 47
img_channels = 1

image_tensor = layers.Input(shape=(img_x, img_y, img_z, img_channels))
encoder_output = residual_encoder(image_tensor)

#from keras.layers import Reshape
#mid=Sequential()
#mid.add(Reshape((3, 4, 3, 100)))
#mid_output = mid(encoder_output)


network_output = residual_decoder(encoder_output)
autoencoder = models.Model(inputs=[image_tensor], outputs=[network_output])
encoder = models.Model(inputs=[image_tensor], outputs=[encoder_output])

print(autoencoder.summary())


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 49, 58, 47, 1 0                                            
__________________________________________________________________________________________________
zero_padding3d_2 (ZeroPadding3D (None, 64, 64, 64, 1 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_107 (Conv3D)             (None, 32, 32, 32, 3 11008       zero_padding3d_2[0][0]           
__________________________________________________________________________________________________
batch_normalization_107 (BatchN (None, 32, 32, 32, 3 128         conv3d_107[0][0]                 
__________________________________________________________________________________________________
leaky_re_l

In [11]:
import numpy as np
x_train=np.load('Data/EMOTION_100_0.2.npy')



In [12]:
mask_img ='utils/mask_152_4mm.nii.gz'
from nilearn.plotting import plot_stat_map, show
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, standardize=True)
masker.fit()

x_train_3D=masker.inverse_transform(x_train) 

In [13]:

x=x_train_3D.get_fdata()
del x_train_3D
x=np.expand_dims(x, axis=5)

x=np.swapaxes(x,2,3)
x=np.swapaxes(x,1,2)
x=np.swapaxes(x,0,1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


In [14]:
x.shape

(3600, 49, 58, 47, 1)

In [15]:
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(x, x,
                epochs=1,
                batch_size=20,
                shuffle=True)

Epoch 1/1
3600/3600 [==============================] - 214s 59ms/step - loss: 0.0137


In [16]:
encoder_output2 =  layers.GlobalAveragePooling3D()(encoder_output)
#encoder_output2 =  Dense()(encoder_output)

encoder1 = models.Model(inputs=[image_tensor], outputs=[encoder_output2])

In [17]:
y=encoder1.predict(x)

In [18]:
from utils.Map_Visualizer import thresholding, plot_maps,gen_map

map1=gen_map(y,x_train)

map1=thresholding(map1)

plot_maps(map1,'Results/ResAE/Emotion1')
np.save('Results/ResAE/Emotion1/ResAE_components.npy',map1)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


Dir created


/usr/local/lib/python3.5/dist-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)
